In [1]:
import os
import pickle

# numpy
import numpy as np
import numpy.ma as ma

from skimage.transform import resize
from math import pi

# plotting
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('default')

# netCDF
import netCDF4 as nc

from tqdm.notebook import tqdm
from tqdm.notebook import trange

import xarray as xr
import xesmf as xe

/home/trevorh2/anaconda3/lib/python3.9/site-packages/esmpy/interface/loadESMF.py:92: VersionWarning: ESMF installation version 8.7.0 beta snapshot, ESMPy version 8.7.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


In [2]:
def get_timeperiod(f):
    rng = f.split('_')[-1][:-3]
    return [s[:-2] for s in rng.split('-')]

In [3]:
def model_info(x):
    _, _, model, _, ens, _, date = x.split('_')
#     date = get_timeperiod(date)
    return [model, ens]

In [4]:
def process_tas(ncdata, var, nlat, nlon):
    _data = np.array(ncdata.variables[var])
    _lats = np.array(ncdata.variables['lat'])
    _lons = np.array(ncdata.variables['lon'])

    grid_in = {"lon": _lons, "lat": _lats}
    grid_out = {"lon": np.linspace(np.min(_lons), np.max(_lons), 180), 
                "lat": np.linspace(np.min(_lats), np.max(_lats), 90)}
    regridder = xe.Regridder(grid_in, grid_out, "bilinear")

    _data = np.array(_data) - 272.15
    _data = regridder(_data)
    _data = np.rot90(_data, k = 2, axes = (1, 2))
    _data = np.flip(_data, axis = 2)
    
    return _data

In [5]:
_path = 'ncdata/hist_max/'
_files = sorted(os.listdir(_path))
_files.reverse()

nlat, nlon = 90, 180

xhist = []
i = 0

model_prev = _files.pop()

ncdata = nc.Dataset(_path + model_prev, "r")
curr_data = process_tas(ncdata, 'tasmax', nlat, nlon)

model_prev = model_info(model_prev)

xhist.append(curr_data)

while _files:
    model_curr = _files.pop()
    
    ncdata = nc.Dataset(_path + model_curr, "r")
    curr_data = process_tas(ncdata, 'tasmax', nlat, nlon)
    
    model_curr = model_info(model_curr)    
    if model_prev == model_curr:
        xhist[i] = np.vstack([xhist[i], curr_data])
    else:
        xhist.append(curr_data)
        i += 1
    
    model_prev = model_curr
    print(model_curr)

['ACCESS-ESM1-5', 'r2i1p1f1']
['CIESM', 'r1i1p1f1']
['CMCC-ESM2', 'r1i1p1f1']
['CNRM-CM6-1', 'r1i1p1f2']
['CNRM-ESM2-1', 'r1i1p1f2']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-

In [6]:
xhist2 = []
for x in xhist:
    if x.shape[0] == 2004:
        xhist2.append(x[:-24])
    else:
        xhist2.append(x)

In [7]:
[x.shape for x in xhist2]

[(1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180),
 (1980, 90, 180)]

In [8]:
xhist = np.stack(xhist2, axis = 1)

In [9]:
[np.max(xhist[:,i]) for i in range(20)], [np.min(xhist[:,i]) for i in range(20)]

([52.34741,
  51.888626,
  71.87354,
  50.724014,
  48.301174,
  50.355854,
  50.575623,
  50.93964,
  51.70199,
  51.161297,
  49.980766,
  50.74135,
  51.311016,
  48.08822,
  47.064384,
  71.75849,
  71.30644,
  54.15121,
  53.370815,
  46.502056],
 [-65.699234,
  -66.33894,
  -62.228806,
  -75.0533,
  -71.088806,
  -70.22678,
  -64.109184,
  -64.93676,
  -71.85775,
  -74.45604,
  -70.99644,
  -71.231514,
  -75.983376,
  -76.32341,
  -77.42581,
  -68.03152,
  -68.81435,
  -65.47919,
  -63.272266,
  -75.25131])

In [10]:
_path = 'ncdata/ssp245_max/'
_files = sorted(os.listdir(_path))
_files.reverse()

xrcp = []
i = 0

model_prev = _files.pop()

ncdata = nc.Dataset(_path + model_prev, "r")
curr_data = process_tas(ncdata, 'tasmax', nlat, nlon)

model_prev = model_info(model_prev)

xrcp.append(curr_data)

while _files:
    model_curr = _files.pop()
    
    ncdata = nc.Dataset(_path + model_curr, "r")
    curr_data = process_tas(ncdata, 'tasmax', nlat, nlon)
    
    model_curr = model_info(model_curr)    
    if model_prev == model_curr:
        xrcp[i] = np.vstack([xrcp[i], curr_data])
    else:
        xrcp.append(curr_data)
        i += 1
    
    model_prev = model_curr
    print(model_curr)

['ACCESS-ESM1-5', 'r2i1p1f1']
['CIESM', 'r1i1p1f1']
['CMCC-ESM2', 'r1i1p1f1']
['CNRM-CM6-1', 'r1i1p1f2']
['CNRM-ESM2-1', 'r1i1p1f2']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r2i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FGOALS-g3', 'r3i1p1f1']
['FIO-ESM-2-0', 'r1i1p1f1']
['FIO-ESM-2-0', 'r2i1p1f1']
['GFDL-ESM4', 'r1i1p1f1']
['GFDL-ESM4', 'r2i1p1f1']
['INM-CM5-0', 'r1i1p1f1']
['IPSL-CM6A-LR', 'r1i1p1f1']
['IPSL-CM6A-LR', 'r2i1p1f1']
['MIROC6', 'r1i1p1f1']
['MIROC6', 'r2i1p1f1']
['MRI-ESM2-0', 'r1i1p1f1']
['MRI-ESM2-0', 'r2i1p1f1']
['MRI-ESM2-0', 'r2i1p1f1']
['NESM3', 'r1i1p1f1']


In [11]:
[x.shape for x in xrcp]

[(1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180),
 (1032, 90, 180)]

In [12]:
xrcp = np.stack(xrcp, axis = 1)

In [13]:
[np.max(xrcp[:,i]) for i in range(20)], [np.min(xrcp[:,i]) for i in range(20)]

([55.998966,
  55.876606,
  75.75146,
  54.42928,
  52.304234,
  53.667316,
  52.950428,
  51.984604,
  54.10825,
  54.128654,
  53.35738,
  54.660496,
  53.076008,
  51.679268,
  51.158104,
  74.141426,
  74.037834,
  56.427223,
  56.55018,
  50.20235],
 [-62.737225,
  -62.69572,
  -59.88652,
  -69.78233,
  -69.687355,
  -68.13495,
  -62.498264,
  -62.331127,
  -68.239105,
  -69.38486,
  -69.797134,
  -69.77264,
  -74.498276,
  -73.49416,
  -74.531364,
  -66.68714,
  -65.93386,
  -62.914543,
  -63.015667,
  -72.33831])

In [14]:
xhist.shape

(1980, 20, 90, 180)

In [15]:
xrcp.shape

(1032, 20, 90, 180)

In [16]:
!touch data/xhist_tasmax_hr.pkl
!touch data/xrcp_tasmax_hr.pkl

In [17]:
pickle.dump(xhist, open("data/xhist_tasmax_hr.pkl", "wb"))
pickle.dump(xrcp, open("data/xrcp_tasmax_hr.pkl", "wb"))